In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import math
import random
from ast import literal_eval

2025-04-07 19:54:43.705618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744055683.718899 3770876 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744055683.723236 3770876 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-07 19:54:43.737620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
class MiniPINN(tf.keras.Model):
    def __init__(self, n_channels, array_geometry, he_initializer = True, floormod=False):
        super().__init__()
        # Fixed metrics
        self.mse_metric = tf.keras.metrics.MeanSquaredError(name='mse')
        self.mae_metric = tf.keras.metrics.MeanAbsoluteError(name='mae')

        # Use floormod
        self.floormod = floormod
       
        # Target array geometry
        self.array_geometry = array_geometry
        
        
        self.initializer = tf.keras.initializers.HeNormal(1561)
        if not he_initializer:
            self.initializer = tf.keras.initializers.GlorotUniform(1561)
        self.net = tf.keras.Sequential([
            tf.keras.layers.Input(n_channels),
            tf.keras.layers.Dense(units=512, activation='relu', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=256, activation='relu', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=128, activation='relu', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=64, activation='relu', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=32, activation='relu', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=8, activation='relu', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=1, activation = 'linear')
        ]) 

    def compile(self, *args, **kwargs):
        super().compile(*args, **kwargs)

    def train_step(self, dataset):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, c, y = dataset
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            self.loss.set_param(x,c)
            loss = self.compute_loss(y=y, y_pred=y_pred)
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
               metric.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}
        
    def test_step(self, dataset):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, c, y = dataset
        y_pred = self(x, training=False)  # Forward pass
        # Compute the loss value
        # (the loss function is configured in `compile()`)
        self.loss.set_param(x,c)
        loss = self.compute_loss(y=y, y_pred=y_pred)
        # Compute gradients
        
        # Update metrics (includes the metric that tracks the loss)
        for metric in self.metrics:
            #print(metric.name)
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def predict(self, inputs,verbose=False):
        output = self.net(inputs)
        if self.floormod:
            #output = tf.math.floormod(output,360.0)
            output = tf.clip_by_value(output,0.0,359.99)
        return output
    def call(self, inputs):
        output = self.net(inputs)
        if self.floormod:
            output = tf.clip_by_value(output,0.0,359.99)
            #output = tf.math.floormod(output,360.0)
        return output

In [14]:
class MiniPINNLoss(tf.keras.losses.Loss):
    # MniPINN Loss introducing physic to the model
    # Taus = R * cos(Theta) / C
    # Taus is time delays
    # R reference for a fixed reference time delay
    # R is the array geometry
    # C is speed
    def __init__(self, target_aa_geo, ref_aa_geo, norm, clambda=0.0):
        super().__init__()
        self.clambda = clambda
        self.r0 = ref_aa_geo
        self.r = target_aa_geo # (num of channels,2)
        self.norm = norm
        self.taus = None
        self.c = None
    def set_param(self, taus, c):
        self.taus = taus
        self.c = c
    def cossin_loss(self, y_actual, y_predicted):
        # convert degree to radian
        actual_rad = y_actual*np.pi/180
        predicted_rad = y_predicted*np.pi/180
        
        # the cosine and sine loss
        cos_loss = 1 - tf.math.cos(actual_rad - predicted_rad)
        sin_loss = tf.math.sin(actual_rad - predicted_rad)
        
        # loss 
        loss = cos_loss + sin_loss
        return loss
    def cos_loss(self, y_actual, y_predicted):
        # convert degree to radian
        actual_rad = y_actual*np.pi/180
        predicted_rad = y_predicted*np.pi/180
        
        # the cosine loss
        loss = 1 - tf.math.cos(actual_rad-predicted_rad)
        return loss
        
    def call(self, y_actual, y_predicted):
        # convert degree to radian
        actual_rad = y_actual*np.pi/180
        predicted_rad = y_predicted*np.pi/180

        # slowness vector (2,None)
        # since only dealing with azimuth
        x_dir = tf.math.cos(predicted_rad) / self.c
        y_dir = tf.math.sin(predicted_rad) / self.c
        slowness = tf.stack([x_dir, y_dir],1) #/ self.c

        # calculate the time delay for the reference channel
        taus_ref = -tf.tensordot(slowness, self.r0, axes=[[1],[0]]) / self.norm
        taus_ref = tf.reshape(taus_ref, [-1,1,1])

        # calculate the time delays from predicted angle
        taus_hat = -tf.tensordot(slowness, self.r, axes = [[1],[1]]) / self.norm
        # shift the predicted time delays to the reference channel
        taus_hat -= taus_ref
        # match the predicted taus dimension
        taus = tf.expand_dims(self.taus, 1)
        
        # calculate the physical loss
        phys_loss = tf.keras.losses.MSE(taus, taus_hat)

        # MSE loss
        #simple_loss = tf.keras.losses.MSE(y_actual, y_predicted)
        #simple_loss = tf.keras.losses.MSE(actual_rad, predicted_rad)
        #cossin_loss = self.cossin_loss(y_actual, y_predicted)
        cos_loss = self.cos_loss(y_actual, y_predicted)

        # Total loss
        loss = tf.reshape(cos_loss, [-1,1]) + self.clambda*phys_loss
        return loss

In [8]:
def selector_loss(y_actual, y_predicted):
    # Select the minimum angle to deal with wrap around problem
    loss = tf.math.minimum(360-tf.math.abs(y_predicted-y_actual), \
                           tf.math.abs(y_predicted-y_actual))
    return loss

In [9]:
def training_evaluation(list_channels, AA_geometry, \
                        inputs, speeds, labels, norm, clambda = 0.0,
                        he_initializer=True, floormod=True, 
                        epochs = 50, plot = False, save_fig = False,verbose=False):
    # trained models
    models = []
    # models losses history
    losses = []
    # evluation scores
    evaluates = []
    # reference channel at channel 0 for time delays
    ref_geometry = tf.constant(AA_geometry[0][:2], dtype=float)

    # train models for each selected channels in the list of channels
    for channels in list_channels:
        # Get the geometry of the channels only in x and y planes
        array_geometry = tf.constant(AA_geometry[channels][:,:2], dtype=float)
        # pack selected channels data into tensorflow dataset
        dataset = DataSetPacker(inputs, speeds, labels, channels)
        
        # split dataset into train, validation, and test dataset
        train_dataset, val_dataset, test_dataset = dataset.split(shuffle=False)
        # Initialize the model
        model = MiniPINN(np.shape(channels), array_geometry, he_initializer, floormod)
        # Compile model
        model.compile(optimizer='adam', loss=MiniPINNLoss(array_geometry, ref_geometry, norm, clambda))#, metrics=['mse','mae'])
        #model.compile(optimizer='adam', loss='mse', metrics=['mse','mae'])
        l = model.fit(train_dataset.batch(32), epochs=epochs, validation_data = val_dataset.batch(32), verbose=verbose)
        # Train model
        #l = model.fit(train_dataset.batch(32), epochs=epochs, verbose=True)
        # Get model score
        eva = EvaluateModel(model, channels, test_dataset)
        eva.evaluate(verbose=True)
        #eva = EvaluateModel.evaluate(model, test_dataset, False)
        if plot:
            if save_fig:
                eva.plot_training(l, save_dir = 'plot/history/')
                eva.plot_evaluation(save_dir = 'plot/evaluation/')
            else:
                eva.plot_training(l)
                eva.plot_evaluation()
        models.append(model)
        losses.append(l)
        evaluates.append(eva)
    return models, evaluates, l
    